In [1]:
#import all required libraries
import random
import pandas as pd
import numpy as np
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [2]:
#reading csv that is placed on my gdrive
#data_set=pd.read_csv('/content/drive/MyDrive/Kaggle Dataset/creditcard.csv')

data_set = pd.read_csv('C:/Users/asama/DL/Completed_Assignments/creditcard.csv')
data_set.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data_set['Class']=data_set['Class'].astype('float64')#converting the only int type column to float

In [5]:
pos_class=data_set[data_set['Class']==1]#selecting all the rows that have Class equals to 1 from Dataset 

In [6]:
neg_class=data_set.sample(pos_class.shape[0])#randomly selecting the Class equals to 0 from Dataset

In [7]:
neg_class.shape,pos_class.shape

((492, 31), (492, 31))

In [8]:
#shuffling the Dataset on basic of even or odd
final=[]
pos=0
neg=0
for i in range(len(pos_class)+len(neg_class)):
  if i%2==0:
    final.append(pos_class.iloc[pos])
    pos+=1
  else:
    final.append(neg_class.iloc[neg])
    neg+=1

In [9]:
final=pd.DataFrame(final)
final.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1.0
199741,133112.0,2.037384,-0.375495,-1.199800,0.594572,-0.391602,-1.091879,-0.040837,-0.200207,1.145114,...,0.107647,0.510953,0.020752,0.081271,0.130060,0.729235,-0.083269,-0.074790,12.31,0.0
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1.0
67198,52408.0,-1.121774,1.046825,1.449851,-0.853597,-0.697472,-0.891311,-0.019569,0.531940,-0.019775,...,-0.095875,-0.278814,0.167095,0.637857,-0.326769,0.659947,0.175923,0.061234,4.13,0.0
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1.0


In [10]:
train_val=int((50/100)*len(final))
test_val=int((30/100)*len(final))
val_val=int((20/100)*len(final))

In [11]:
#separating the test train and validation data
train_data=final.iloc[:train_val , :-1]
train_labels=final.iloc[:train_val,-1]

val_data=final.iloc[train_val :train_val+val_val , :-1]
val_labels=final.iloc[train_val :train_val+val_val,-1]

test_data=final.iloc[train_val+val_val: , :-1]
test_labels=final.iloc[train_val+val_val: ,-1]

In [12]:
mean=train_data.mean(axis=0)
std=train_data.std(axis=0)

train_data-=mean
val_data-=mean
test_data-=mean

train_data/=std
val_data/=std
test_data/=std

In [13]:
model= models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(30,)))
model.add(layers.Dense(8, activation="relu"))
model.add(layers.Dense(6,activation="relu"))

model.add(layers.Dense(1,activation="sigmoid"))

model.compile(optimizer="adam", 
              loss="binary_crossentropy", 
              metrics=["accuracy"])

In [14]:
model.fit(train_data,train_labels,
          validation_data=(val_data,val_labels),
          batch_size=4,
          epochs=100)

Epoch 1/100
123/123 [==============================] - 1s 3ms/step - loss: 0.6835 - accuracy: 0.5513 - val_loss: 0.5898 - val_accuracy: 0.7755
Epoch 2/100
123/123 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 0.8303 - val_loss: 0.3620 - val_accuracy: 0.8622
Epoch 3/100
123/123 [==============================] - 0s 2ms/step - loss: 0.2667 - accuracy: 0.9150 - val_loss: 0.2877 - val_accuracy: 0.8827
Epoch 4/100
123/123 [==============================] - 0s 1ms/step - loss: 0.1950 - accuracy: 0.9258 - val_loss: 0.2542 - val_accuracy: 0.8980
Epoch 5/100
123/123 [==============================] - 0s 1ms/step - loss: 0.1485 - accuracy: 0.9503 - val_loss: 0.2251 - val_accuracy: 0.8929
Epoch 6/100
123/123 [==============================] - 0s 1ms/step - loss: 0.1405 - accuracy: 0.9465 - val_loss: 0.2087 - val_accuracy: 0.9031
Epoch 7/100
123/123 [==============================] - 0s 1ms/step - loss: 0.1518 - accuracy: 0.9492 - val_loss: 0.2024 - val_accuracy: 0.9031

Epoch 58/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0235 - accuracy: 0.9928 - val_loss: 0.3534 - val_accuracy: 0.9388
Epoch 59/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0108 - accuracy: 0.9974 - val_loss: 0.3454 - val_accuracy: 0.9337
Epoch 60/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0082 - accuracy: 0.9985 - val_loss: 0.3613 - val_accuracy: 0.9388
Epoch 61/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 0.9970 - val_loss: 0.3740 - val_accuracy: 0.9337
Epoch 62/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0156 - accuracy: 0.9953 - val_loss: 0.3731 - val_accuracy: 0.9337
Epoch 63/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0080 - accuracy: 0.9984 - val_loss: 0.3797 - val_accuracy: 0.9286
Epoch 64/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0163 - accuracy: 0.9909 - val_loss: 0.3857 - val_accuracy:

In [15]:
model.evaluate(test_data,test_labels)

10/10 [==============================] - 0s 1ms/step - loss: 2.4114 - accuracy: 0.7939


[2.4113731384277344, 0.7939189076423645]